In [3]:
'''
import pymysql
db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong')
cur = db.cursor()
company='京东'
title='测试标题'
href='测试网址'
date='测试数据'
source='测试来源'
sql = 'insert into test(company,title,href,date,source) values (%s,%s,%s,%s,%s)'
cur.execute(sql,(company,title,href,date,source))
db.commit()
cur.close()
db.close()
'''

In [119]:
#import spider
#import python_database
import requests
import re

#爬取不同游戏很多页
import requests
import re
import time
def baidu(company):
    #num = (page-1)*10
    url = 'https://www.baidu.com/s?ie=utf-8&;cl=2&medium=0&rtt=4&bsst=1&rsv_dl=news_t_sk&tn=news&word=' + company+'&pn=0'
    headers={'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
    res = requests.get(url,headers=headers,timeout=10).text
    p_href = '<h3 class="c-title">.*?<a href="(.*?)"'
    p_title = '<h3 class="c-title">.*?>(.*?)</a>'
    p_info = '<p class="c-author">(.*?)</p>'
    href = re.findall(p_href,res,re.S)
    title = re.findall(p_title,res,re.S)
    info = re.findall(p_info,res,re.S)
    #print('ok1')
    
    #score
    score=[]
    keywords = ['违约','诉讼','兑付','垃圾','退游','花钱']
    for i in range(len(title)):
        num = 0
        try:
            headers={'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
            article = requests.get(href[i],headers=headers,timeout=10).text
            #print(article)
        except:
            article = '单个爬取失败'
            
        try:
            article = article.encode('ISO-8859-1').decode('utf-8')
        except:
            try:
                article = article.encode('ISO-8859-1').decode('gbk')
            except:
                article = article
        p_article = '<p>(.*?</p>)'
        article_main = re.findall(p_article,article)
        article = '.join(article_main)'
        #print("ok2")
        
        
        
        for k in keywords:
            if (k in article) or (k in title[i]):
                num -= 5
        score.append(num)
        #print("ok3")
    
    
    #数据清洗
    source=[]
    date=[]
    for i in range(len(title)):
        print(len(title))
        title[i] = title[i].strip()
        title[i] = re.sub('<.*?>','',title[i])
        info[i] = re.sub('<.*?>','',info[i])
    #print(info[i])
        source.append(info[i].split('&nbsp;&nbsp;')[0])
        date.append(info[i].split('&nbsp;&nbsp;')[1])
        source[i] = source[i].strip()
        #print("ok4")
        
        date[i] = date[i].strip()
        #date[i] = data[i].split(' ')[0]
        date[i] = re.sub('年','-',date[i])
        date[i] = re.sub('月','-',date[i])
        date[i] = re.sub('日','-',date[i])
        #print('date')
        
        if ('小时' in date[i]) or ('分钟' in date[i]):
            date[i] = time.strftime("%Y-%m-%d")
        else:
            date[i] = data[i]
        
        print(str(i+1)+'.'+title[i]+'('+date[i]+'-'+source[i]+')')
        print(href[i])
        print(company + '该条舆情评分为' + str(score[i]))
        #print(url)
    #for i in range(len(title)):
        #file1 = open('./data.txt','a')
        #file1.write(str(i+1)+'.'+title[i]+'('+date[i]+'-'+source[i]+')'+'\n')
        #file1.write(href[i]+'\n')
        #file1.write('-------------------------------------------------------------------------------------------'+ '\n'+'\n')
        #file1.close()

    return title, href, date, source, score

import pymysql
class database_execute(object):
    """
        数据库操作类
    """
    def __init__(self, db_name):
        self.db_name = db_name

    def connect_db(self):
        db = pymysql.connect(host='localhost', port=3308, user='root', password='xm221010',
                                  database=self.db_name, charset='utf8')
        return db

    def insert_db(self, company, title, href, date, source):
        db = self.connect_db()
        cur = db.cursor()
        sql = 'INSERT INTO test(company, title, href, date, source) VALUES (%s, %s, %s, %s, %s)'
        cur.execute(sql, (company, title, href, date, source))
        db.commit()
        cur.close()
        db.close()

def main():

    companys= ['王者荣耀','神仙道','和平精英','三国志','阴阳师']
    title = []
    href = []
    date = []
    source = []
    score = []
          
    for company in companys:
        try:
            title, href, date, source, score = baidu(company)
            print(len(title))
            for i in range(len(title)):
                #time
                #print('start')
                '''
                date[i] = data[i].split(' ')[0]
                date[i] = re.sub('年','-',date[i])
                date[i] = re.sub('月','-',date[i])
                date[i] = re.sub('日','-',date[i])
                print('date')
                if ('小时' in date[i]) or ('分钟' in date[i]):
                    date[i] = to,e.striftime("%Y-%m-%d")
                else:
                    date[i] = data[i] 
                #time
                '''
                
                #噪音过滤-标题#
                
                if company not in title[i]:
                    title[i] = ''
                    href[i] = ''
                    date[i] = ''
                    source[i] = ''
                    score[i] = ''
                    print("null")
                
   
                
                #噪音过滤-正文#
                '''
                try:
                    headers={'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
                    article = requests.get(href[i],headers=headers,timeout=10).text
                    #print(article)
                except:
                    article = '爬取失败'
                    print('nok')
                company_re = company[0] + '.{0,5}' + company[-1]
                if len(re.findall(company_re,article)) < 1:
                    print('article')
                    title[i] = ''
                    href[i] = ''
                    date[i] = ''
                    source[i] = ''
                    score[i] = ''
                    #print("null")
                
                
                    
            while '' in title:
                title.remove('')
            while '' in href:
                href.remove('')
            while '' in date:
                date.remove('')
            while '' in source:
                source.remove('')
            while '' in score:
                score.remove('')
                print("remove")
                '''
                
                
                db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong')
                cur = db.cursor()
                sql2='select * from article where company = %s'
                cur.execute(sql2,company)
                data_all=cur.fetchall()
                title_all=[]
                #print('ready')
                for j in range(len(data_all)):
                    title_all.append(data_all[j][1])
                    #print('ok')
                if title[i] not in title_all:
                    #print('doing')
                    sql3 = 'insert into article(company,title,href,date,source,score) values (%s,%s,%s,%s,%s,%s)'
                    cur.execute(sql3,(company,title[i],href[i],date[i],source[i],score[i]))
                    db.commit()
                    cur.close()
                    db.close()
                    print(company + '百度新闻爬取成功！')
                else:
                    print("same")
        except:
            print(company + '百度新闻爬取失败！')
            

    

if __name__ == '__main__':
    main()

10
date
1.王者荣耀:蒙恬上线三天,暴击打野流强势崛起,飞牛:细节很关键(2020-06-05-手机游戏网)
http://www.sjyx.com/news/686037.html
王者荣耀该条舆情评分为0
10
date
2.王者荣耀:TS和XQ一决胜负,为何都看好TS?观众调侃因为工资!(2020-06-05-迅游游戏)
https://baijiahao.baidu.com/s?id=1668624534224411661&amp;wfr=spider&amp;for=pc
王者荣耀该条舆情评分为0
10
date
3.王者荣耀:婉儿预售有细节,顶部1行小字,等等党直言等不下去!(2020-06-05-游戏阿呆)
https://baijiahao.baidu.com/s?id=1668624373804537109&amp;wfr=spider&amp;for=pc
王者荣耀该条舆情评分为0
10
date
4.热血沸腾!陈奕天秀特技演唱《王者荣耀》(2020-06-05-长城网)
http://news.hebei.com.cn/system/2020/06/05/100336989.shtml
王者荣耀该条舆情评分为0
10
date
5.王者荣耀k头意思介绍(2020-06-05-18183手游网)
https://www.18183.com/yxzjol/202006/2879008.html
王者荣耀该条舆情评分为0
10
date
6.王者荣耀S20赛季什么时候开启 王者荣耀S19赛季结束时间(2020-06-05-18183手游网)
https://www.18183.com/yxzjol/202006/2879211.html
王者荣耀该条舆情评分为0
10
date
7.王者荣耀:上官婉儿上分攻略,大招一划就断,掌握这三点拒绝断大(2020-06-05-腾讯网)
https://new.qq.com/omn/20200605/20200605A04OSD00.html
王者荣耀该条舆情评分为0
10
date
8.王者荣耀日暮之流什么时候上线_日暮之流上线时间介绍(2020-06-05-apk8安卓网)
https://www.apk8.com/zixun/54070_1.html
王者荣耀该条舆情评分为0
10
date
9.王

In [1]:
#爬取不同游戏很多页
import requests
import re
import time
def baidu(company,page):
    num = (page-1)*10
    url = 'https://www.baidu.com/s?ie=utf-8&;cl=2&medium=0&rtt=4&bsst=1&rsv_dl=news_t_sk&tn=news&word=' + company+'&pn='+str(num)
    headers={'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
    res = requests.get(url,headers=headers,timeout=10).text
    p_href = '<h3 class="c-title">.*?<a href="(.*?)"'
    p_title = '<h3 class="c-title">.*?>(.*?)</a>'
    p_info = '<p class="c-author">(.*?)</p>'
    href = re.findall(p_href,res,re.S)
    title = re.findall(p_title,res,re.S)
    info = re.findall(p_info,res,re.S)
    #print('ok1')
    
    #score
    score=[]
    keywords = ['违约','诉讼','兑付','垃圾','退游','花钱']
    for i in range(len(title)):
        num = 0
        try:
            headers={'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
            article = requests.get(href[i],headers=headers,timeout=10).text
            #print(article)
        except:
            article = '单个爬取失败'
            
        try:
            article = article.encode('ISO-8859-1').decode('utf-8')
        except:
            try:
                article = article.encode('ISO-8859-1').decode('gbk')
            except:
                article = article
        p_article = '<p>(.*?</p>)'
        article_main = re.findall(p_article,article)
        article = '.join(article_main)'
        #print("ok2")
        
        
        
        for k in keywords:
            if (k in article) or (k in title[i]):
                num -= 5
        score.append(num)
        #print("ok3")
    
    
    #数据清洗
    source=[]
    date=[]
    for i in range(len(title)):
        print(len(title))
        title[i] = title[i].strip()
        title[i] = re.sub('<.*?>','',title[i])
        info[i] = re.sub('<.*?>','',info[i])
    #print(info[i])
        source.append(info[i].split('&nbsp;&nbsp;')[0])
        date.append(info[i].split('&nbsp;&nbsp;')[1])
        source[i] = source[i].strip()
        #print("ok4")
        
        date[i] = date[i].strip()
        #date[i] = data[i].split(' ')[0]
        date[i] = re.sub('年','-',date[i])
        date[i] = re.sub('月','-',date[i])
        date[i] = re.sub('日','-',date[i])
    print('date')
        
        if ('小时' in date[i]) or ('分钟' in date[i]):
            date[i] = time.strftime("%Y-%m-%d")
        else:
            date[i] = data[i]
        
        print(str(i+1)+'.'+title[i]+'('+date[i]+'-'+source[i]+')')
        print(href[i])
        print(company + '该条舆情评分为' + str(score[i]))
        #print(url)
    #for i in range(len(title)):
        #file1 = open('./data.txt','a')
        #file1.write(str(i+1)+'.'+title[i]+'('+date[i]+'-'+source[i]+')'+'\n')
        #file1.write(href[i]+'\n')
        #file1.write('-------------------------------------------------------------------------------------------'+ '\n'+'\n')
        #file1.close()

    return title, href, date, source, score

import pymysql
class database_execute(object):
    """
        数据库操作类
    """
    def __init__(self, db_name):
        self.db_name = db_name

    def connect_db(self):
        db = pymysql.connect(host='localhost', port=3308, user='root', password='xm221010',
                                  database=self.db_name, charset='utf8')
        return db

    def insert_db(self, company, title, href, date, source):
        db = self.connect_db()
        cur = db.cursor()
        sql = 'INSERT INTO test(company, title, href, date, source) VALUES (%s, %s, %s, %s, %s)'
        cur.execute(sql, (company, title, href, date, source))
        db.commit()
        cur.close()
        db.close()

def main():

    companys= ['王者荣耀','神仙道','和平精英','三国志','阴阳师']
    title = []
    href = []
    date = []
    source = []
    score = []
    print("ready1")
          
    for company in companys:
        for page in range(30):
            try:
                title, href, date, source, score = baidu(company,page)
                print(len(title))
                for i in range(len(title)):
                    #time
                    print('start')                                     
                    db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong')
                    cur = db.cursor()
                    sql2='select * from article where company = %s'
                    cur.execute(sql2,company)
                    data_all=cur.fetchall()
                    title_all=[]
                    print('ready')
                    for j in range(len(data_all)):
                        title_all.append(data_all[j][1])
                        #print('ok')
                    if title[i] not in title_all:
                        print('doing')
                        sql3 = 'insert into article(company,title,href,date,source,score) values (%s,%s,%s,%s,%s,%s)'
                        cur.execute(sql3,(company,title[i],href[i],date[i],source[i],score[i]))
                        db.commit()
                        cur.close()
                        db.close()
                        print(company + '百度新闻爬取成功！')
                    else:
                        print("same")
            except:
                print(company + '百度新闻爬取失败！')
            

    

if __name__ == '__main__':
    main()

ready1
10
1.王者荣耀:天美技穷了?新英雄刚出就买一送一,超值英雄合集(2020-06-05-Aggro君)
https://baijiahao.baidu.com/s?id=1668629616130888356&amp;wfr=spider&amp;for=pc
王者荣耀该条舆情评分为0
10
2.王者荣耀开黑欢乐多,和平精英刚枪超刺激,二合一就成了这款神作(2020-06-05-南爵电竞)
https://baijiahao.baidu.com/s?id=1668629307768537483&amp;wfr=spider&amp;for=pc
王者荣耀该条舆情评分为0
10
3.苹果和安卓能一起王者荣耀吗(2020-06-05-18183手游网)
https://www.18183.com/gonglue/202006/2879542.html
王者荣耀该条舆情评分为0
10
4.王者荣耀阿古朵技能怎么样?大招可召唤强力宠物[多图](2020-06-05-173手游快爆)
https://www.11773.com/xinwen/246166.html
王者荣耀该条舆情评分为0
10
5.王者荣耀改不了名字怎么办-改不了名字原因介绍(2020-06-05-爪游控)
https://www.zhuayoukong.com/news/201635.html
王者荣耀该条舆情评分为0
10
6.王者荣耀改名卡为什么下架 王者荣耀改名功能暂未开放(2020-06-05-琵琶网)
http://wy.pipaw.com/xinwen/news502935.html
王者荣耀该条舆情评分为0
10
7.王者荣耀杨玉环羞花皮肤怎么样_王者荣耀杨玉环羞花皮肤介绍(2020-06-05-apk8安卓网)
https://www.apk8.com/zixun/54089_1.html
王者荣耀该条舆情评分为0
10
8.王者荣耀蒙恬国服最强攻略教学 蒙恬铭文出装及连招打法详解(2020-06-05-中华网游戏频道)
https://game.china.com/zt/jyyz/13003853/20200605/38309687.html
王者荣耀该条舆情评分为0
10
9.王者荣耀:蒙恬上线三天,暴击打野流强势崛起,飞牛:细节很关键(2020-06-05-手机游戏网

In [108]:
import pymysql
import time 
db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong',charset='utf8')
company='王者荣耀'
#today=time.strftime("%Y-%m-%d")
cur = db.cursor()
sql2='select * from article where company = %s'
cur.execute(sql2,company)
data=cur.fetchall()
score = 100
for i in range(len(data)):
    score+=int(data[i][5])
db.commit()
cur.close()
db.close()
print(company+'的今日舆情评分为'+str(score))

王者荣耀的今日舆情评分为100


In [120]:
#baidu('王者荣耀')

In [121]:
#date

In [123]:
import pymysql
import datetime
import pandas as pd
db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong',charset='utf8')
company='王者荣耀'
date_list = list(pd.date_range('2020-06-01','2020-06-05')) #时间区间
for i in range(len(date_list)):
    date_list[i] = datetime.datetime.strftime(date_list[i],'%Y-%m-%d')
cur = db.cursor()
sql='select * from article where company = %s and date = %s'
score_list={}
for d in date_list:
    cur.execute(sql,(company,d))
    data=cur.fetchall()
    score = 100
    for i in range(len(data)):
        score+=int(data[i][5])
    score_list[d] = score
db.commit()
cur.close()
db.close()

In [127]:
score_list
data = pd.DataFrame.from_dict(score_list,orient='index',columns=['score'])
data.index.name = 'date'
data

,score
date,
2020-06-01,100
2020-06-02,100
2020-06-03,100
2020-06-04,100
2020-06-05,100


In [76]:
#Tushare

In [2]:
import tushare as ts
df = ts.get_hist_data('000002',start='2019-01-01',end='2020-06-04')

In [11]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

user = '495271301@qq.com'
pwd = 'dspnrlqkrosecbbd'
to = '495271301@qq.com'

db = pymysql.connect(host='localhost',port=3306,user='root',password='xm221010',database='pachong',charset='utf8')
companys= ['王者荣耀','神仙道','和平精英','三国志','阴阳师']
today=time.strftime("%Y-%m-%d")
cur = db.cursor()
sql='select * from article where company = %s'
data_all = []
for c in companys:
    cur.execute(sql,c)
    data=cur.fetchall()
    for j in range(len(data)):
        data_all.append(data[j])
db.commit()
cur.close()
db.close()

mail_msg = []
mail_msg.append('<p style="margin:0 auto">尊敬的用户，您好，以下是今天的舆情监控报告，望查阅：</p>')
mail_msg.append('<p><b>一、王者荣耀舆情监控报告</b></p>')
for i in range(len(data_all)):
    href = '<p><a href="'+data_all[i][2]+'">' + data_all[i][1] + ':score=' + data_all[i][5] + '</a></p>'
    mail_msg.append(href)
mail_msg.append('<p>祝好</p>')
mail_msg.append('<p>林旺腾</p>')
mail_msg = '\n'.join(mail_msg)
print(mail_msg)

msg = MIMEText(mail_msg,'html','utf-8')
msg['Subject'] = '测试邮件主题3'
msg['From'] = user
msg['To'] = to
s = smtplib.SMTP_SSL('smtp.qq.com',465)
s.login(user,pwd)
s.send_message(msg)
s.quit()
print('Success!')

<p>尊敬的用户，您好，以下是今天的舆情监控报告，望查阅：</p>
<p><b>一、王者荣耀舆情监控报告</b></p>
<p><a href="https://new.qq.com/omn/20200604/20200604A0FGUV00.html">王者荣耀:阿古朵曝光或成首个射坦英雄,李白典藏皮肤上线成谜:score=0</a></p>
<p><a href="https://new.qq.com/omn/20200604/20200604A0JG2500.html">王者荣耀:新英雄阿古朵太过可爱,瑶的萌宠地位就此不保?:score=0</a></p>
<p><a href="https://new.qq.com/omn/20200604/20200604A06Y7J00.html">王者荣耀:焦点比赛即将来临!AG对战DYG,这次连久哲也来了!:score=0</a></p>
<p><a href="https://baijiahao.baidu.com/s?id=1668581379181352918&amp;wfr=spider&amp;for=pc">王者荣耀:现版本为数不多的中核,上官婉儿该如何带动节奏!:score=0</a></p>
<p><a href="https://baijiahao.baidu.com/s?id=1668581335893258384&amp;wfr=spider&amp;for=pc">王者荣耀:墙体机制改动上线测评,达摩和女娲才是最大赢家:score=0</a></p>
<p><a href="https://www.18183.com/yxzjol/202006/2878566.html">王者荣耀上官婉儿天狼绘梦者6折特权怎么获得:score=0</a></p>
<p><a href="https://www.18183.com/yxzjol/202006/2878197.html">王者荣耀杨玉环羞花特效什么样 王者荣耀杨玉环四美特效曝光:score=0</a></p>
<p><a href="http://www.hxnews.com/news/dmyx/djyx/yxhb/202006/04/1901425.shtml">王者荣耀天狼绘梦者体验卡怎么获得 上官婉儿KPL皮肤体验卡领取方法:score=0

In [7]:
                   data_all=cur.fetchall()
                    title_all=[]
                    print('ready')
                    for j in range(len(data_all)):
                        title_all.append(data_all[j][1])
                        #print('ok')

IndentationError: unexpected indent (<ipython-input-7-fa8a6848c770>, line 2)